In [1]:
import boto3
import pandas as pd
import json
from io import BytesIO

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Initialize a session using boto3
session = boto3.Session(region_name='us-east-1')

# Create an S3 client
s3_client = session.client('s3')

# Specify your bucket name and the key (path) to the JSON file
bucket_name = 'idvera-backend-dev'
#file_key = 'data/data-0303633f-7870-4ea3-9291-0eeac521f18a.json'
file_key = 'data/data-024837cc-34b1-4116-ab8f-7e3a28bccb24.json'


In [3]:
# Get the JSON object from S3 directly into memory
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
content = response['Body'].read()

# Decode bytes to string and load JSON into a Python dictionary
data = json.loads(content.decode('utf-8'))

# Flatten the nested structures using json_normalize
df = pd.json_normalize(data)

In [4]:
df.head()

,sessionId,deviceId,deviceType,endTimestamp,startTimestamp,device_id,magnetometerData.rawData,magnetometerData.calibratedData,magnetometerData.unit,magnetometerData.timestamp,accelerometerData.unit,accelerometerData.timestamp,accelerometerData.rawData,accelerometerData.calibratedData,barometerData.rawData,barometerData.unit,barometerData.rawAbsoluteData,barometerData.timestamp,barometerData.calibratedData,barometerData.rawRelativeData,gyroscopeData.rawData,gyroscopeData.calibratedData,gyroscopeData.timestamp,gyroscopeData.unit
0,0E9F3F69-3A89-4A03-A8BA-7EBB023F0233,2C18CFCB-23A3-4B87-9127-0908D9D63BF5,iPhone,7.417974e+08,7.417974e+08,5c0e03fbe0f20d6deb12200679d829ecf3ecfe8932286d...,"[{'timestamp': 741797370.492727, 'x': 106.0643...","[{'z': 64.93824693853206, 'y': 49.894166356867...",μT (microtesla),7.417974e+08,value * 9.8 meters/second,7.417974e+08,"[{'z': -1.000335693359375, 'timestamp': 741797...","[{'z': -0.37165421919389197, 'timestamp': 7417...","[{'pitch': 0.009049265635124872, 'yaw': -1.898...",kPA,"[{'altitude': 929.0288391113281, 'accuracy': 1...",7.417974e+08,"[{'timestamp': 741797376.55711, 'yaw': -1.8978...","[{'timestamp': 741797370.49564, 'relativeAltit...","[{'z': 0.0036634444259107113, 'y': 0.001907089...","[{'z': 0.00044450554944143553, 'x': -8.7303264...",7.417974e+08,radians/second


In [5]:
# Replace with your actual file path and name
#file_path = 'data-0303633f-7870-4ea3-9291-0eeac521f18a.json'

# Load JSON file into a Python dictionary
#with open(file_path, 'r') as f:
#    data = json.load(f)
    
# Flatten the nested structures using json_normalize
#df = pd.json_normalize(data)

In [6]:
#data

In [7]:
# Explode or expand 'magnetometerData'
df_magnetometerData = pd.json_normalize(data['magnetometerData'], record_path=['calibratedData'], meta=['timestamp', 'unit'], meta_prefix='magnetometer').add_suffix('_calibratedData_m')
df_magnetometerData_raw = pd.json_normalize(data['magnetometerData'], record_path=['rawData'], meta=['timestamp', 'unit'], meta_prefix='magnetometer').add_suffix('_rawData_m')

# Explode or expand 'barometerData'
df_barometerData = pd.json_normalize(data['barometerData'], record_path=['calibratedData'], meta=['timestamp', 'unit'], meta_prefix='barometerData').add_suffix('_calibratedData_b')
df_barometerData_raw = pd.json_normalize(data['barometerData'], record_path=['rawData'], meta=['timestamp', 'unit'], meta_prefix='barometerData').add_suffix('_rawData_b')
df_barometerData_rawRelativeData = pd.json_normalize(data['barometerData'], record_path=['rawRelativeData'], meta=['timestamp', 'unit'], meta_prefix='barometerData').add_suffix('_rawRelativeData_b')
df_barometerData_rawAbsoluteData = pd.json_normalize(data['barometerData'], record_path=['rawAbsoluteData'], meta=['timestamp', 'unit'], meta_prefix='barometerData').add_suffix('_rawAbsoluteData_b')


# Explode or expand 'gyroscopeData'
df_gyroscopeData = pd.json_normalize(data['gyroscopeData'], record_path=['calibratedData'], meta=['timestamp', 'unit'], meta_prefix='gyroscopeData').add_suffix('_calibratedData_g')
df_gyroscopeData_raw = pd.json_normalize(data['gyroscopeData'], record_path=['rawData'], meta=['timestamp', 'unit'], meta_prefix='gyroscopeData').add_suffix('_rawData_g')

# Explode or expand 'accelerometerData'
df_accelerometerData = pd.json_normalize(data['accelerometerData'], record_path=['calibratedData'], meta=['timestamp', 'unit'], meta_prefix='accelerometerData').add_suffix('_calibratedData_a')
df_accelerometerData_raw = pd.json_normalize(data['accelerometerData'], record_path=['rawData'], meta=['timestamp', 'unit'], meta_prefix='accelerometerData').add_suffix('_rawData_a')


In [8]:
# Concatenate the DataFrames
df_total = pd.concat([df_magnetometerData, df_magnetometerData_raw,
                      df_barometerData, df_barometerData_raw, df_barometerData_rawRelativeData, df_barometerData_rawAbsoluteData,
                      df_gyroscopeData, df_gyroscopeData_raw,
                      df_accelerometerData, df_accelerometerData_raw], axis=1)

In [9]:
df_total.head()

,z_calibratedData_m,y_calibratedData_m,x_calibratedData_m,timestamp_calibratedData_m,magnetometertimestamp_calibratedData_m,magnetometerunit_calibratedData_m,timestamp_rawData_m,x_rawData_m,y_rawData_m,z_rawData_m,magnetometertimestamp_rawData_m,magnetometerunit_rawData_m,timestamp_calibratedData_b,yaw_calibratedData_b,pitch_calibratedData_b,roll_calibratedData_b,barometerDatatimestamp_calibratedData_b,barometerDataunit_calibratedData_b,pitch_rawData_b,yaw_rawData_b,roll_rawData_b,timestamp_rawData_b,barometerDatatimestamp_rawData_b,barometerDataunit_rawData_b,timestamp_rawRelativeData_b,relativeAltitude_rawRelativeData_b,pressure_rawRelativeData_b,barometerDatatimestamp_rawRelativeData_b,barometerDataunit_rawRelativeData_b,altitude_rawAbsoluteData_b,accuracy_rawAbsoluteData_b,timestamp_rawAbsoluteData_b,precision_rawAbsoluteData_b,barometerDatatimestamp_rawAbsoluteData_b,barometerDataunit_rawAbsoluteData_b,z_calibratedData_g,x_calibratedData_g,y_calibratedData_g,timestamp_calibratedData_g,gyroscopeDatatimestamp_calibratedData_g,gyroscopeDataunit_calibratedData_g,z_rawData_g,y_rawData_g,x_rawData_g,timestamp_rawData_g,gyroscopeDatatimestamp_rawData_g,gyroscopeDataunit_rawData_g,z_calibratedData_a,timestamp_calibratedData_a,x_calibratedData_a,y_calibratedData_a,accelerometerDatatimestamp_calibratedData_a,accelerometerDataunit_calibratedData_a,z_rawData_a,timestamp_rawData_a,x_rawData_a,y_rawData_a,accelerometerDatatimestamp_rawData_a,accelerometerDataunit_rawData_a
0,64.938247,49.894166,57.822250,7.417974e+08,741797399.114796,μT (microtesla),7.417974e+08,106.064316,17.186264,55.050583,741797399.114796,μT (microtesla),7.417974e+08,-1.897858,0.011167,-0.011569,741797398.882393,kPA,0.009049,-1.898401,-0.008015,7.417974e+08,741797398.882393,kPA,7.417974e+08,0.000000,91.158005,741797398.882393,kPA,929.028839,17.071457,7.417974e+08,0.5,741797398.882393,kPA,0.000445,-0.000087,0.000062,7.417974e+08,741797399.582662,radians/second,0.003663,0.001907,-0.001293,7.417974e+08,741797399.582662,radians/second,-0.371654,7.417974e+08,-0.006431,-0.009292,741797398.882237,value * 9.8 meters/second,-1.000336,7.417974e+08,-0.017899,-0.011780,741797398.882237,value * 9.8 meters/second
1,77.539144,67.057999,81.891567,7.417974e+08,741797399.114796,μT (microtesla),7.417974e+08,106.064316,17.186264,55.050583,741797399.114796,μT (microtesla),7.417974e+08,-1.897845,0.011160,-0.011538,741797398.882393,kPA,0.009983,-1.898257,-0.009656,7.417974e+08,741797398.882393,kPA,7.417974e+08,0.034363,91.157616,741797398.882393,kPA,929.044037,17.071457,7.417974e+08,0.5,741797398.882393,kPA,0.000524,0.000071,0.000156,7.417974e+08,741797399.582662,radians/second,0.003663,0.001907,-0.001293,7.417974e+08,741797399.582662,radians/second,-0.475618,7.417974e+08,-0.155051,-0.128540,741797398.882237,value * 9.8 meters/second,-1.000336,7.417974e+08,-0.017899,-0.011780,741797398.882237,value * 9.8 meters/second
2,85.016966,77.175315,96.108321,7.417974e+08,741797399.114796,μT (microtesla),7.417974e+08,106.064316,17.186264,55.050583,741797399.114796,μT (microtesla),7.417974e+08,-1.897872,0.011199,-0.011630,741797398.882393,kPA,0.010369,-1.898181,-0.010351,7.417974e+08,741797398.882393,kPA,7.417974e+08,0.071350,91.157227,741797398.882393,kPA,929.064514,17.071457,7.417974e+08,0.5,741797398.882393,kPA,0.000662,0.000160,0.000310,7.417974e+08,741797399.582662,radians/second,0.001243,0.000626,0.000452,7.417974e+08,741797399.582662,radians/second,-0.547888,7.417974e+08,-0.260055,-0.212732,741797398.882237,value * 9.8 meters/second,-1.006790,7.417974e+08,-0.016434,-0.011169,741797398.882237,value * 9.8 meters/second
3,89.732906,83.553606,105.065390,7.417974e+08,741797399.114796,μT (microtesla),7.417974e+08,105.836884,17.510880,54.938675,741797399.114796,μT (microtesla),7.417974e+08,-1.897780,0.011291,-0.011665,741797398.882393,kPA,0.010656,-1.897974,-0.010671,7.417974e+08,741797398.882393,kPA,7.417974e+08,0.071350,91.157227,741797398.882393,kPA,929.058578,17.071457,7.417974e+08,0.5,74179739

In [10]:
df_total.shape

(2251, 59)

In [11]:
df_total['relativeAltitude_rawRelativeData_b'].head()

0    0.000000
1    0.034363
2    0.071350
3    0.071350
4    0.050232
Name: relativeAltitude_rawRelativeData_b, dtype: float64

In [12]:
# Extracting and mapping relevant columns to mimic the original dataframe structure
mapped_df = pd.DataFrame({
    'latitude': None,  # Needs to be derived or obtained separately
    'longitude': None,  # Needs to be derived or obtained separately
    'elevation': df_total['relativeAltitude_rawRelativeData_b'],
    'magnetic_field': df_total['x_calibratedData_m'] + df_total['y_calibratedData_m'] + df_total['z_calibratedData_m'],
    'gravity_anomaly': df_total['x_rawData_a'] + df_total['y_rawData_a'] + df_total['z_rawData_a'],
    'pressure': df_total['altitude_rawAbsoluteData_b']
})

In [13]:
mapped_df.head(40)

,latitude,longitude,elevation,magnetic_field,gravity_anomaly,pressure
0,None,None,0.000000,172.654663,-1.030014,929.028839
1,None,None,0.034363,226.488710,-1.030014,929.044037
2,None,None,0.071350,258.300603,-1.034393,929.064514
3,None,None,0.071350,278.351902,-1.028366,929.058578
4,None,None,0.050232,291.769134,-1.026718,929.030838
5,None,None,0.050232,301.250096,-1.032562,928.993439
6,None,None,0.050232,308.253516,-1.034210,928.967255
7,None,None,0.010620,313.622029,-1.037399,928.960892
8,None,None,-0.015808,317.866614,-1.031937,928.976990
9,None,None,-0.036987,321.312307,-1.029419,929.016617
